In [1]:
OUT_DIR = '/tmp/'
NUM_WORKERS = 16
BATCH_SIZE = 512

# Barebones starter example

### Imports

In [2]:
from robustness import model_utils, datasets, train, defaults
from robustness.datasets import CIFAR
import torch as ch

# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
from cox.utils import Parameters
import cox.store

/home/eecs/utrerf/anaconda3/lib/python3.7/site-packages/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


### Make dataset and loaders

In [3]:
# Hard-coded dataset, architecture, batch size, workers
ds = CIFAR('/tmp/')
m, _ = model_utils.make_and_restore_model(arch='resnet18', dataset=ds)
train_loader, val_loader = ds.make_loaders(batch_size=BATCH_SIZE, workers=NUM_WORKERS)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


### Make a cox store for logging

In [4]:
# Create a cox store for logging
out_store = cox.store.Store(OUT_DIR)

Logging in: /tmp/e2d03a23-6778-4c3a-b200-105180b9ec99


### Set up training arguments

In [5]:
# Hard-coded base parameters
train_kwargs = {
    'out_dir': "train_out",
    'adv_train': 1,
    'constraint': '2',
    'eps': 0.5,
    'attack_lr': 0.1,
    'attack_steps': 7,
    'epochs': 5,
    'log_iters'  : 10,
    'stop_probability' : 0.1
}
train_args = Parameters(train_kwargs)

# Fill whatever parameters are missing from the defaults
train_args = defaults.check_and_fill_args(train_args,
                        defaults.TRAINING_ARGS, CIFAR)
train_args = defaults.check_and_fill_args(train_args,
                        defaults.PGD_ARGS, CIFAR)

### Train Model

In [6]:
# Train a model
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)
pass

Train Epoch:0 | Loss 2.1885 | AdvPrec1 24.386 | AdvPrec5 77.284 | Reg term: 0.0 ||: 100%|██████████| 98/98 [00:45<00:00,  2.15it/s]
Val Epoch:0 | Loss 1.6648 | NatPrec1 37.850 | NatPrec5 88.830 | Reg term: 0.0 ||: 100%|██████████| 20/20 [00:01<00:00, 10.85it/s]
Val Epoch:0 | Loss 2.1446 | AdvPrec1 19.450 | AdvPrec5 78.660 | Reg term: 0.0 ||: 100%|██████████| 20/20 [00:49<00:00,  2.49s/it]
Train Epoch:1 | Loss 1.6106 | AdvPrec1 39.524 | AdvPrec5 90.064 | Reg term: 0.0 ||: 100%|██████████| 98/98 [00:45<00:00,  2.17it/s]
Train Epoch:2 | Loss 1.4020 | AdvPrec1 48.324 | AdvPrec5 92.950 | Reg term: 0.0 ||: 100%|██████████| 98/98 [00:46<00:00,  2.10it/s]
Train Epoch:3 | Loss 1.2554 | AdvPrec1 54.114 | AdvPrec5 94.506 | Reg term: 0.0 ||:  32%|███▏      | 31/98 [00:15<00:34,  1.95it/s]

KeyboardInterrupt: 

# Customizations

## Custom loss

In [ ]:
train_crit = ch.nn.CrossEntropyLoss()
def custom_train_loss(logits, targ):
    probs = ch.ones_like(logits) * 0.5
    logits_to_multiply = ch.bernoulli(probs) * 9 + 1
    return train_crit(logits_to_multiply * logits, targ)

adv_crit = ch.nn.CrossEntropyLoss(reduction='none').cuda()
def custom_adv_loss(model, inp, targ):
    logits = model(inp)
    probs = ch.ones_like(logits) * 0.5
    logits_to_multiply = ch.bernoulli(probs) * 9 + 1
    new_logits = logits_to_multiply * logits
    return adv_crit(new_logits, targ), new_logits

train_args.custom_train_loss = custom_train_loss
train_args.custom_adv_loss = custom_adv_loss

In [ ]:
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)

## Custom data loaders

### Using LambdaLoader

In [ ]:
from robustness.loaders import LambdaLoader

def label_noiser(ims, labels):
    label_noise = ch.randint_like(labels, high=9)
    probs = ch.ones_like(label_noise) * 0.1
    labels_to_noise = ch.bernoulli(probs.float()).long()
    new_labels = (labels + label_noise * labels_to_noise) % 10
    return ims, new_labels

train_loader = LambdaLoader(train_loader, label_noiser)

In [ ]:
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)
pass

### Using TransformedLoader

In [ ]:
from robustness.loaders import TransformedLoader
from robustness.data_augmentation import TRAIN_TRANSFORMS_DEFAULT

def make_rand_labels(ims, targs):
    new_targs = ch.randint(0, high=10,size=targs.shape).long()
    return ims, new_targs

train_loader_transformed = TransformedLoader(train_loader,
                                            make_rand_labels,
                                            TRAIN_TRANSFORMS_DEFAULT(32),
                                            workers=8,
                                            batch_size=BATCH_SIZE,
                                            do_tqdm=True)

In [ ]:
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)
pass

## Custom per-iteration logging

In [ ]:
CUSTOM_SCHEMA = {'iteration': int, 'weight_norm': float }
out_store.add_table('custom', CUSTOM_SCHEMA)

In [ ]:
from torch.nn.utils import parameters_to_vector as flatten

def log_norm(mod, it, loop_type, inp, targ):
    if loop_type == 'train':
        curr_params = flatten(mod.parameters())
        log_info_custom = { 'iteration': it,
                    'weight_norm': ch.norm(curr_params).detach().cpu().numpy() }
        out_store['custom'].append_row(log_info_custom)
    
train_args.iteration_hook = log_norm

In [ ]:
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)
pass

## Custom architecture

In [ ]:
from torch import nn
from robustness.model_utils import make_and_restore_model

class MLP(nn.Module):
    # Must implement the num_classes argument
    def __init__(self, num_classes=10):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 1000)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(1000, num_classes)

    def forward(self, x, *args, **kwargs):
        out = x.view(x.shape[0], -1)
        out = self.fc1(out)
        out = self.relu1(out)
        return self.fc2(out)

new_model = MLP(num_classes=10)

In [ ]:
new_model, _ = make_and_restore_model(arch=new_model, dataset=ds)

In [ ]:
train.train_model(train_args, new_model, (train_loader, val_loader), store=out_store)
pass